# Computational biology and bioinformatics - <span style="color:#1CA766">INFO-F-439</span>
# Assignment 1: <span style="color:#1CA766">GOR III secondary structure prediction</span>

> ## <span style="color:#2E66A7"> Alberto Parravicini</span>

*****

# <span style="color:#2E66A7">Part 1:</span> Introduction

Proteins are for the most part defined by the aminoacid sequences that compose them. However, due to the nature of the aminoacids, proteins have more complex structures that characterize them.

Hydrogen bonds between the aminoacids in the sequence causes the proteins to assume three-dimensional structures of various kind. Being able to recognize these structures is useful to compare proteins, and to get a better insight on their properties.

Classifying correctly which portion of a protein assumes a given secondary structure isn't an easy task, but it can be seen that there are links between specific patterns of aminoacids and given secondary structures.
As an example, in $\alpha$-helices (one type of secondary structure) every third aminoacid along the sequence will tend to be hydrophobic. Also, regions richer of *alanine (A)* and *leucine (L)* will tend to form helices. (see *Gollery, Martin. "Bioinformatics: Sequence and Genome Analysis", David W. Mount. Pag. 387-388*)

***

### <span style="color:#1CA766">GOR I & II</span>:

One idea that can be exploited to predict secondary structures from aminoacid sequeces, is that the secondary structure corresponding to a certain aminoacid is influenced not only by the aminoacid itself, but also by the aminoacids that are close to it.

Given a dataset of proteins where for each aminoacid its secondary structure is known, it is possible to build a model that is able to predict secondary sequences, by using the idea above.

This is the core idea behind the **GOR I** and **GOR II** algorithms.

For a given position $j$, it will be predicted the secondary structure $S_j$ that has the highest value of *information*, as function of the aminoacid sequence.
Assuming that our secondary structures are in the set $S = \{\alpha$-helices (**H**), $\beta$-sheets (**B**), and coils (**C**)$\}$, the one that will be chosen is $$S_j = \arg\max_{S}{I(\Delta S_j;\ a_1, \ldots, a_n)}$$ where $a_1, \ldots, a_n$ is the aminoacid sequence, and $S_j$ is picked among the secondary structures in S.

The previous computation is approximated by **GOR** by considering a window of size $17$ around position $j$, which gives the formula $$S_j = \arg\max_{S}{I(\Delta S_j;\ a_1, \ldots, a_n)} \approx \arg\max_{S}{\sum_{m = -8}^{m = 8}{I(\Delta S_j;\ a_{j+m})}}$$ 

In the previous formulation, $I(\Delta S_j;\ a_{j+m})$ is a measure of *self-information*, and is given by
$$I(\Delta S_j;\ a_{j+m}) = log(\frac{c_{S_j,\ a_{j+m}}}{c_{\neg S_j,\ a_{j+m}}}) + log(\frac{c_{\neg S}}{c_{S}})$$

where $c$ refers to the count of the occurrences (in a given dataset) of what is specified in the subscript; for instance, $c_{\neg S}$ is the number of aminoacids with a secondary structure different from $S$.

In Python, we can compute $c_{S_j,\ a_{j+m}}$ and $I(\Delta S_j;\ a_{j+m})$ in the following way:

In [4]:
def count_residues_gor_1(s_list, a_list, s_j, a_jm, lag):
    """
    Count the number of times that structure "s_j" appears,
    and residue "a_jm" is at distance "lag" from them.
    :param s_list: a list of secondary structures; pandas Series.
    :param a_list: a list of aminoacids; pandas Series.
    :param s_j: a secondary structure name.
    :param a_jm: an aminoacid name.
    :param lag: distance from a_j at which a_jm is searched.
    :return: int
    """
    n = len(s_list)
    count = 0
    for i in range(max(0, -lag), min(n, n - lag)):
        if s_list.iloc[i] == s_j and a_list.iloc[i + lag] == a_jm:
            count += 1
    return count

In [5]:
def info_value_gor_1(s_list, a_list, s_j, a_jm, lag, a_occ):
    """
    Compute the information value associated to a secondary structure s_j, in position j,
    and to an aminoacid a_jm, at distance m from s_j.
    :param s_list: a list of secondary structures; pandas Series.
    :param a_list: a list of aminoacids; pandas Series.
    :param s_j: a secondary structure name.
    :param a_jm: an aminoacid name.
    :param lag: distance from a_j at which a_jm is searched.
    :param a_occ: number of occurrencies of each aminoacid; pandas Series.
    :return: double
    """
    num_1 = count_residues_gor_1(s_list, a_list, s_j, a_jm, lag)
    den_1 = a_occ[a_jm] - num_1
    den_2 = s_occ[s_j]
    num_2 = len(s_list) - den_2
    return np.log(num_1 * num_2) - np.log(den_1 * den_2)

***

### <span style="color:#1CA766">GOR III</span>:

It is possible to improve the accuracy of **GOR I & II** by considering not only the occurrencies of aminoacids close to the position where we want to predict the secondary structure, but also their occurrencies relatively to the aminoacid found at that given position.

This means that the information value of a secondary structure $S_j$ will be approximated as 

$$S_j = \arg\max_{S}{I(\Delta S_j;\ a_1, \ldots, a_n)} \approx \arg\max_{S}{\sum_{m = -8}^{m = 8}{I(\Delta S_j;\ a_{j+m}\ |\ a_j)}} = \ldots$$ 
$$\ldots = I(\Delta S_j;\ a_j) + \sum_{m = -8,\ m \neq 0}^{m = 8}{I(\Delta S_j;\ a_{j+m}\ |\ a_j)}$$

where it holds $$I(\Delta S_j;\ a_{j+m}\ |\ a_j) = log(\frac{c_{S_j,\ a_{j+m},\ a_j}}{c_{\neg S_j,\ a_{j+m},\ a_j}}) + log(\frac{c_{\neg S_j,\ a_j}}{c_{S_j,\ a_j}})$$

In Python, we can compute $c_{S_j,\ a_{j+m},\ a_j}$ and $I(\Delta S_j;\ a_{j+m}\ |\ a_j)$ as:

In [6]:
def count_residues_gor_3(s_list, a_list, s_j, a_j, a_jm, lag):
    """
    Count the number of times that structure "s_j" appears together with residue "a_j",
    and residue "a_jm" is at distance "lag" from them.
    :param s_list: a list of secondary structures; pandas Series.
    :param a_list: a list of aminoacids; pandas Series.
    :param s_j: a secondary structure name
    :param a_j: an aminoacid name
    :param a_jm: an aminoacid name
    :param lag: distance from a_j at which a_jm is searched
    :return: int
    """
    n = len(s_list)
    count = 0
    for i in range(max(0, -lag), min(n, n - lag)):
        if s_list.iat[i] == s_j and a_list.iat[i] == a_j and a_list.iat[i + lag] == a_jm:
            count += 1
    return count

In [7]:
def info_value_gor_3(s_list, a_list, s_j, a_j, a_jm, lag, a_occ):
    """
    Compute the information value associated to an aminoacid a_j in position j,
    with secondary structure s_j, in position j,
    and to an aminoacid a_jm, at distance m from s_j.
    :param s_list: a list of secondary structures; pandas Series.
    :param a_list: a list of aminoacids; pandas Series.
    :param s_j: a secondary structure name
    :param a_j: an aminoacid name
    :param a_jm: an aminoacid name
    :param lag: distance from a_j at which a_jm is searched
    :param a_occ: number of occurrencies of each aminoacid
    :return: double
    """
    # Num of times s_j appears with a_j, and a_jm is lag position distant.
    num_1 = count_residues_gor_3(s_list, a_list, s_j, a_j, a_jm, lag)
    # Num of times a_j appears with a secondary structure different from s_j,
    # and a_jm is lag position distant.
    den_1 = count_aminoacid_lag(a_list, a_j, a_jm, lag) - num_1
    
    # Number of times aminoacid a_j appears together with structure s_j.                        
    den_2 = count_residues_gor_3(s_list, a_list, s_j, a_j, a_j, 0)                          
    # Number of times aminoacid a_j appears with a different structure from s_j.
    num_2 = a_occ[a_j] - den_2
                 
    info = np.log(num_1) - np.log(den_1) + np.log(num_2) - np.log(den_2)   
    return info     

*****

# <span style="color:#2E66A7">Part 2:</span> Data pre-processing

To compute the information values for each combination of aminoacids and secondary structures, it is necessary to have a large dataset of proteins with known secondary structures. We are provided with a set of 498 proteins, where the secondary structures have been determined by **DSSP** and **STRIDE**.

Before being able to apply **GOR III**, it is required to preprocess the dataset, clean the inconsistencies that might be present, and store the aminoacid sequences in an appropriate way.

It can be noted that there are a few rows with aminoacids that are not part of the usual 20. As the number of these rows is very limited compared to the total, we can just remove them entirely, and this won't affect the overall results of the algorithm.

In [8]:
#%% IMPORT DATA

import pandas as pd
import numpy as np
import timeit
import pickle

aminoacid_list = "arndceqghilkmfpstwyv"

# Dictionary that maps aminoacid codes to single letters.
aminoacid_codes = {"ala":  "a",
                   "arg":  "r",
                   "asn":  "n",
                   "asp":  "d",
                   "cys":  "c",
                   "gln":  "q",
                   "glu":  "e",
                   "gly":  "g",
                   "his":  "h",
                   "ile":  "i",
                   "leu":  "l",
                   "lys":  "k",
                   "met":  "m",
                   "phe":  "f",
                   "pro":  "p",
                   "ser":  "s",
                   "thr":  "t",
                   "trp":  "w",
                   "tyr":  "y",
                   "val":  "v"}

In [38]:
def preprocess_input(file_name, aminoacid_codes):
    """
    Load and preprocess a file containing protein sequences.
    The output will be a DataFrame ready to be used by GOR.
    :param file_name: string, name of the file to be opened.
    :param aminoacid_codes: dictionary that maps aminoacid codes to single letters.
    :return: DataFrame
    """
    # Load the data
    input_data = pd.read_csv(file_name,
                         header=None, sep="\t",
                         names=["PDB_code", "PDB_chain_code", "PDB_seq_code", "residue_name", "secondary_structure"])

    # CLEAN DATA - AMINOACIDS

    # There are some values that are weird, like "a", "b", ...
    # Some of those can be interpreted as correct aminoacids.

    # Remove rows with "X", "b", "UNK"
    # First, remove any leading spaces, then remove "X", "b", "UNK"
    input_data["residue_name"] = input_data["residue_name"].str.strip()
    input_data = input_data.drop(input_data[input_data["residue_name"].isin(["X", "b", "UNK"])].index).reset_index(drop=True)


    # Add a column for the aminoacids, while preserving the original one
    input_data["a"] = input_data["residue_name"]
    input_data["a"] = input_data["a"].str.lower()


    # replace the codes with something shorter
    input_data["a"] = input_data["a"].map(lambda x: aminoacid_codes[x] if x in aminoacid_codes else x)

    # Create a new column
    input_data["s"] = input_data["secondary_structure"]
    input_data["s"] = input_data["s"].str.lower()
    # Replace "other" with "coil"
    input_data.loc[input_data["s"] == "other", "s"] = "coil"
    # Shorten values
    input_data["s"] = input_data["s"].str[0]
    
    # Add a composite key to the dataset.
    # Append the PDB chain code to the PDB code, to obtain unique protein identifier.
    input_data["PDB_code_and_chain"] = input_data.PDB_code + "_" + input_data.PDB_chain_code

    return input_data


We can load and work on the *STRIDE* dataset. The *DSSP* dataset is similar, and will be compared to *STRIDE* later in the report. 

In [39]:
    # Type of the data to read ("stride", "dssp")
    data_type = "stride"
    # File name
    file_name = "../data/" + data_type + "_info.txt"
    # Read the data
    input_data = preprocess_input(file_name, aminoacid_codes)

    # Look at the aminoacid values
    aminoacids = set(input_data.residue_name)
    print(aminoacids)
    # Same stuff with the secondary structures
    secondary_structures = set(input_data.secondary_structure)
    print(secondary_structures)

{'SER', 'ASP', 'ASN', 'ALA', 'GLU', 'LYS', 'ILE', 'GLY', 'PRO', 'PHE', 'ARG', 'MET', 'VAL', 'TYR', 'TRP', 'HIS', 'LEU', 'GLN', 'THR', 'CYS'}
{'Helix', 'Other', 'Beta'}


In [40]:
input_data.iloc[0:5, ]

,PDB_code,PDB_chain_code,PDB_seq_code,residue_name,secondary_structure,a,s,PDB_code_and_chain
0,1w0n,A,12,ILE,Other,i,c,1w0n_A
1,1w0n,A,13,THR,Beta,t,b,1w0n_A
2,1w0n,A,14,LYS,Beta,k,b,1w0n_A
3,1w0n,A,15,VAL,Beta,v,b,1w0n_A
4,1w0n,A,16,GLU,Beta,e,b,1w0n_A


**PDB_code** and **PDB_chain_code** work as identifier of the row. The column **a** and **s** are just shortened versions of the aminoacid codes and of the secondary structure codes.

We can also compute some other values, that will be useful later.

In [19]:
# Number of residues
n_res = len(input_data.index)

# Compute the number of occurrencies of each aminoacid
a_occ = input_data["a"].value_counts()

# Compute the number of occurrencies of each secondary structure
s_occ = input_data["s"].value_counts()

In [22]:
n_res

71073

In [20]:
a_occ

l    6449
a    5963
g    5119
v    4960
e    4816
d    4244
k    4213
s    4148
i    4034
t    3937
r    3665
p    3181
n    3053
f    2890
q    2849
y    2467
h    1715
m    1260
c    1103
w    1007
Name: a, dtype: int64

In [21]:
s_occ

c    29170
h    25755
b    16148
Name: s, dtype: int64

Now, to compute the information values of **GOR III** it is required to add a few more functions.

The first will compute a matrix that stores, for each aminoacid, the occurrences of each secondary structure.  
The second will build a dictionary where each key corresponds to a secondary structure, and each value is a tensor (technically, a pandas.Panel) that stores the number of times where an aminoacid $a_j$ appears with secondary structure $S_j$,
and another aminoacid $a_{j+m}$ appears at a distance $m$ from them, with $m$ in $[-8, 8]$.

It is necessary to precompute these matrices as counting from scratch the occurrencies, for each prediction, could be very inefficient.

In [27]:
def build_sj_aj_matrix(s_list, a_list):
    """
    Count the number of times that each aminoacid a_j has secondary structure s_j.
    :param s_list: a list of secondary structures; pandas Series.
    :param a_list: a list of aminoacids; pandas Series.
    :return: pandas.DataFrame
    """
    # Set of all secondary structures.
    sec_structure_set = set(s_list)
    # Set of all aminoacids.
    aminoacid_set = set(a_list)
    # Occurrency matrix.
    sj_aj_matrix = pd.DataFrame(0, index=sec_structure_set, columns=aminoacid_set, dtype=int)
    # Zip together the lists of secondary structures and aminoacids, it makes counting easier.
    s_a_list = [list(x) for x in zip(s_list,a_list)]
    # Count the occurrencies.
    for s in sec_structure_set:
        for a in aminoacid_set:
            sj_aj_matrix.at[s, a] = s_a_list.count([s, a])

    return sj_aj_matrix

In [28]:
start_time = timeit.default_timer()
sj_aj_matrix = build_sj_aj_matrix(input_data.s, input_data.a)
end_time = timeit.default_timer()
print("! -> EXECUTION TIME OF build_sj_aj_matrix:", (end_time - start_time), "\n")

! -> EXECUTION TIME OF build_sj_aj_matrix: 0.30610202826584043 



In [29]:
sj_aj_matrix

,p,a,v,e,h,c,f,s,n,d,y,l,r,q,t,m,w,g,k,i
b,335,1007,2115,773,400,301,938,771,447,530,827,1573,742,493,1135,282,294,804,809,1572
h,467,2992,1601,2348,570,305,1012,1204,916,1338,850,3071,1586,1369,1089,547,377,850,1795,1468
c,2379,1964,1244,1695,745,497,940,2173,1690,2376,790,1805,1337,987,1713,431,336,3465,1609,994


In [30]:
# Count the times where aminoacid a_j appears with secondary structure s_j,
# and aminoacid a_jm appears at a distance m from them, with m in [-8, 8].
#   WARNING: this can take a bit of time, usually about 20-30 minutes;
#            do it at your own risk!
start_time = timeit.default_timer()
##########################
# UNCOMMENT IF NEEDED ####
##########################
# sj_aj_ajm_dict = build_sj_aj_ajm(input_data)

end_time = timeit.default_timer()
print("! -> EXECUTION TIME OF build_sj_aj_ajm:", (end_time - start_time), "\n")

! -> EXECUTION TIME OF build_sj_aj_ajm: 4.389741792465429e-05 



As the computation of the dictionary can take a fair bit of time, it is a good idea to store the result, and load it when needed.

Note: comment/uncomment the following lines depending on your needs!

In [32]:
# Save data
dict_file_name = "sj_aj_ajm_dict_"+ data_type + ".p"

# # Open the file for writing.
# file_object = open(dict_file_name.encode('utf-8').strip(), 'wb')

# # Save data
# pickle.dump(sj_aj_ajm_dict, file_object)
#
# file_object.close()

Data can also be loaded with:

In [33]:
# Load data
file_object = open(dict_file_name.encode('utf-8').strip(), 'rb')
sj_aj_ajm_dict = pickle.load(file_object)

In [34]:
sj_aj_ajm_dict["b"].iloc[0, :, :]

,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8
r,45,27,56,42,35,47,45,34,742,31,40,38,36,35,42,35,38
p,36,39,41,47,35,42,29,19,0,25,31,37,43,34,48,34,46
i,35,32,32,39,38,46,46,69,0,68,46,39,38,43,41,35,35
y,21,38,23,24,23,31,37,43,0,37,32,25,24,21,18,29,29
w,11,9,6,10,9,8,9,19,0,13,7,8,4,15,13,19,11
e,45,39,40,42,45,42,55,39,0,26,56,43,39,48,53,48,52
m,7,13,14,11,14,19,8,23,0,8,13,10,12,8,17,10,13
h,19,12,16,17,17,23,17,22,0,19,19,17,19,19,16,28,20
c,4,6,8,15,10,10,9,12,0,14,10,14,12,10,12,12,8
v,60,56,48,45,55,64,69,93,0,104,63,77,55,62,38,51,41
